### **Resources**

<hr>

[Project_Inspiration](https://www.youtube.com/watch?v=honnJp7-cCU&t=691s)


## **Tomorrow**

[Auth_Client](https://www.youtube.com/watch?v=honnJp7-cCU&t=691s)


## **NextJs Concepts I Need to Learn**

<hr>

### **Routing**

### **Working of Parameter Catching**

### **Complete Working of Dynamic Routing**

### **Working of API Routes**

### **Complete Working of NextJs for Each Route**

### **Difference Between Client and Server Side Rendering / Client Component and Server Component**

### **Why Async/Await is Used in NextJs to Capture Parameters**


<hr>
<hr>
<hr>
<hr>


## **How to Actually Structure and Start any Full Stack Project**

Whenever you start a new project, we should not just jump into coding. Instead, we should take a step back and think about the project as a whole. This means considering the architecture, the technologies we will use, and how everything will fit together.

<hr>

**First, we need to think about the architecture of the project. This includes the overall structure of the code, how the different components will interact, and how data will flow through the application. We should also consider how we will handle authentication, authorization, and data storage.**

**Next, find reusable components that we can use in our project. This could be libraries, frameworks, or even just code snippets that we have used in previous projects. By reusing code, we can save time and effort, and also ensure that our project is built on a solid foundation.**

**Setup all the images, icons, fonts, colors, and other assets that we will use in the project. This includes any logos, icons, or other images that we want to include, as well as any custom fonts or colors that we want to use. We should also consider how we will handle accessibility, so that our application is usable by everyone.**

**Decide the width, height and layout for the UI components. This includes the overall layout of the application, as well as the specific dimensions and positions of each component. We should also consider how we will handle different screen sizes, so that our application looks good on all devices.**

**Write all the TailwindCSS classes that we will need, Make use of `@apply` to create reusable classes. Write all the classes required one by one, and then combine them into reusable classes using `@apply`. This will help us keep our CSS organized and maintainable. We should also consider how we will handle responsive design, so that our application looks good on all devices.**

**Prepare all the files and folders that we will need for the project. This includes the overall structure of the code, as well as any specific files or folders that we want to include. We should also consider how we will handle version control, so that we can easily track changes to the code over time.**

**Make the project structured and organized. This means considering how we will handle different components, how we will structure the code, and how we will handle data flow. We should also consider how we will handle testing, so that we can ensure that our application is working correctly.**


<hr>
<hr>
<hr>
<hr>


## **Essential Concepts To Master**

<hr>

## **React & Next.js: Key Concepts You Must Master**

### **Core React Concepts**

1. **JSX and Component Rendering**

   - Functional vs Class Components
   - Props and State
   - React rendering cycle

2. **React Reconciliation & Virtual DOM**

   - Diffing algorithm
   - When and how React updates the DOM

3. **State Management**

   - useState, useReducer
   - Context API vs Redux, Zustand, Jotai

4. **Side Effects and Lifecycle**

   - useEffect, cleanup functions
   - Component lifecycle in functional components

5. **Refs and DOM Manipulation**

   - useRef, forwardRef
   - Imperative handle with `useImperativeHandle`

6. **Memoization**

   - useMemo, useCallback
   - React.memo

7. **React Keys and Lists**

   - Efficient rendering of dynamic content

---

### **Next.js-Specific Concepts**

1. **Hydration (and Dehydration)**

   - Difference between SSR-rendered HTML and interactive client-side React
   - Issues like content mismatch (`Text content did not match`)
   - Use `suppressHydrationWarning` wisely
   - Delaying hydration: `useEffect`, dynamic imports (`ssr: false`)

2. **Pre-rendering: SSR vs SSG**

   - getStaticProps, getServerSideProps
   - ISR (Incremental Static Regeneration)
   - Tradeoffs between SSG and SSR

3. **Routing & Dynamic Routes**

   - File-based routing
   - `pages/` vs `app/` directory (App Router)
   - Catch-all routes, dynamic nested routes

4. **App Router (New in Next.js 13+)**

   - Server components vs Client components
   - Layouts and templates
   - Parallel routes and loading UI

5. **Data Fetching Strategies**

   - Fetching on client vs server
   - Caching (next/image, fetch cache, revalidation)
   - Edge functions and middleware

6. **Middleware and Edge Runtime**

   - Modify requests/responses at the edge
   - Use cases like auth, geo-targeting

7. **API Routes**

   - Creating serverless functions
   - Auth, database access, form submission

8. **Environment Variables & Configuration**

   - next.config.js
   - `process.env`, build-time vs runtime configs

9. **Image Optimization**

   - next/image
   - Lazy loading, blur placeholders

10. **Fonts and Global Styles**

    - CSS Modules, TailwindCSS
    - Optimizing font loading (next/font)

11. **Error Handling & Boundaries**

    - React error boundaries
    - next/error
    - `notFound()` and `error()` in App Router

### **Performance, Optimization, and Dev Tools**

1. **Code Splitting and Dynamic Imports**

   - Dynamic imports (`next/dynamic`)
   - Tree-shaking

2. **Bundle Analysis**

   - next-bundle-analyzer
   - Reducing third-party dependencies

3. **Hydration Strategies**

   - Partial hydration
   - Streaming and React Suspense

4. **Progressive Enhancement**

   - Build UIs that work before JavaScript loads

5. **Accessibility (a11y)**

   - Semantic HTML
   - Keyboard navigation
   - ARIA roles

6. **Security**

   - XSS and SSRF in server-rendered apps
   - Environment variable leaks
   - Auth tokens & cookie security

---

### **Testing and Dev Experience**

1. **Unit Testing**

   - Jest, React Testing Library

2. **E2E Testing**

   - Playwright, Cypress

3. **Linting & Formatting**

   - ESLint, Prettier

4. **Types and Type Safety**

   - TypeScript in React and Next.js
   - Props typing, generic components

---

### **Real-World Practices**

1. **Authentication**

   - Auth.js (next-auth), JWT, OAuth

2. **Authorization**

   - Role-based, permission-based UI

3. **Form Handling**

   - React Hook Form, Zod validation

4. **Working with Databases**

   - Prisma, Drizzle ORM

5. **CI/CD & Deployment**

   - Vercel, Netlify, Docker

6. **Monitoring & Logging**

   - Sentry, LogRocket


## **Notes Related to This Project**

<hr>

Create the `Next.js` app with the following command:

```bash
npx create-next-app@latest

```

Then, we set up the project folder structure as follows:

## **Folder Structure**

### **public**

The `public` folder will contain the static files, such as images and fonts.

### **fonts**

The `fonts` folder will contain custom fonts. Inside the `fonts` folder, we've a typescript file `fonts.ts` that exports the custom fonts.

```typescript
import localFont from "next/font/local";

export const satoshi = localFont({
  src: [
    {
      path: "./Satoshi-Light.woff2",
      weight: "300",
      style: "normal",
    },...
  ],
  variable: "--font-satoshi",
});
```

The above code, loads the custom font files i.e. `Satoshi-Light.woff2`, `Satoshi-Regular.woff2`, etc from the local filesystem and generates a optimized `CSS` `@font-face` rule for each font file. The `variable` property allows us to use the font as a CSS variable.

The `@font-face` rules generated by the `next/font/local` package will be automatically injected into the global CSS of the application, making the fonts available for use throughout the app as below:

```css
@font-face {
  font-family: "__satoshi";
  src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
  font-weight: 400;
  font-style: normal;
}
```

The `@font-face` is a CSS rule that allows us to define custom fonts that can be loaded and used in the application. The `font-family` is set to a unique name (in this case, `__satoshi`), which can be used in the stylesheets to apply the font.

As, the users won't have the `fonts` written in the CSS file, so we use the `next/font/local` package to load the fonts. This package optimizes the loading of the fonts by generating the necessary CSS rules and making them available globally in the application.

Also, `CSS` variables are created for each font, allowing us to use them in our stylesheets like this:

```css
.font_satoshi_variable_abc123 {
  --font-satoshi: "__satoshi";
}
```

Also, `fonts` are preloaded to avoid flashing and layout shifts when the fonts are loaded. This is done by adding the `preload` attribute to the `<link>` tag in the `_document.js` file.

```bash
<link rel="preload" href="/_next/static/media/Satoshi-Regular.woff2" as="font" type="font/woff2" crossorigin="anonymous" />
```

Also, `fonts` are hashed and served from `/_next/static/media/` to ensure that the browser caches them correctly and serves the correct version of the font files.

### **Build Time for Fonts**

You're **very close**, and your understanding is almost entirely correct. Let's walk through it step by step and **fine-tune** a few important points to make your mental model crystal clear.

> "After the build we will have an optimized build file..."

Yes! At **build time**, Next.js:

- Processes the font files (like `Satoshi-Regular.woff2`)
- Optimizes them (e.g., hashes file names)
- Generates `@font-face` rules
- Outputs them into the final CSS
- Generates `<link rel="preload" ...>` tags

> "...when the user requests it, the HTML gets downloaded and `<link rel="preload" ...>` runs and downloads the fonts..."

Yes! The **browser parses the HTML**, sees the `<link rel="preload" ...>` tag, and starts downloading the font **early**—before it's even needed—**to reduce render delays**.

Example:

```html
<link
  rel="preload"
  href="/_next/static/media/Satoshi-Regular.woff2"
  as="font"
  type="font/woff2"
  crossorigin="anonymous"
/>
```

This ensures **fast and smooth rendering** (no flashing text).

> "Then after download, the `@font-face` will have the link for the downloaded font and it will use it?"

🟡 **Almost right**, with a small clarification:

- The browser sees your CSS, finds this:

  ```css
  @font-face {
    font-family: "__satoshi";
    src: url("/_next/static/media/Satoshi-Regular.woff2") format("woff2");
    font-weight: 400;
    font-style: normal;
  }
  ```

- Now it knows:

  > "If something uses `font-family: __satoshi`, I should use this file at this weight/style."

- Then, it finds elements styled like:

  ```css
  html.font_satoshi_variable_xyz {
    --font-satoshi: "__satoshi";
  }

  body {
    font-family: var(--font-satoshi), sans-serif;
  }
  ```

- So the browser:

  1. Sees that `--font-satoshi` equals `__satoshi`
  2. Knows `@font-face` defines what `__satoshi` points to
  3. Uses the preloaded font file that’s already been downloaded
  4. Renders the text with `Satoshi`

**Timeline Recap:**

1. **Build time**:

   - Font files hashed and copied to `/_next/static/media/...`
   - `@font-face` CSS is generated
   - `link rel="preload"` is added to the HTML

2. **Client request**:

   - HTML is parsed
   - Fonts are downloaded early via `<link rel="preload">`
   - CSS is parsed, `@font-face` tells how to use those fonts
   - CSS variables like `--font-satoshi` are resolved
   - Text renders instantly using the font

**Note**

After the build, there will not be a single `CSS` file rather there will be multiple CSS files generated for each page and component. This is because Next.js uses **CSS Modules** and **Code Splitting** to optimize the loading of styles. Each page or component will have its own CSS file, which will be loaded only when that page or component is rendered.

Also, there will not be a single `HTML` file rather there will be multiple `HTML` files generated for each page. This is because Next.js uses **Dynamic Routing** to generate the HTML for each page based on the route. But depends on the rendering strategy used, the HTML files can be static or dynamic.

### **app**

The `app` folder will contain the main application code, including pages, components, and styles. Inside the `app` folder, we have the following structure:

```bash
app/
├── global.css
├── layout.tsx
├── page.tsx

```

**global.css**

The `global.css` file will contain global styles for the application. Here, we've

```Css

@theme inline {
  --font-karla: var(--font-geist-karla);
  --font-satoshi: var(--font-satoshi);
  --color-background: var(--background);
  --color-foreground: var(--foreground);
}
```

Which sets up the theme variables for fonts and colors. Once the `browser` loads the `global.css`, it will have access to these variables and can use them in the stylesheets. All the `fonts` would have been downloaded and preloaded by the time the `global.css` is loaded, so the browser will not flash or shift the layout when the fonts are applied.

**layout.tsx**

`import { Karla } from "next/font/google";` loads the `Karla` font from Google Fonts during the build time. This means that the font will be available in the application without needing to load it from an external source at runtime and all the `@font-face` rules will be generated and injected into the global CSS.

```typescript
import { Karla } from "next/font/google";

const karla = Karla({
  subsets: ["latin"],
  weight: ["400", "700"],
});

export default function Layout({ children }: { children: React.ReactNode }) {
  return (
    <html lang="en" className={karla.className}>
      <body>{children}</body>
    </html>
  );
}
```

So, when we do as below:

```typescript
const karla = Karla({
  subsets: ["latin"],
  weight: ["400", "700"],
  variable: "--font-karla",
});
```

We will get an object as:

```typescript
karla = {
  className: "karla_class_xyz",
  variable: "karla_variable_xyz",
};
```

Then use it in the `html` tag as:

```typescript
<html lang="en" className={karla.className}>
```


# **React Notes**

<hr>

This section contains notes related to React concepts, Next.js features, and best practices for building modern web applications. It serves as a quick reference guide for developers working with React and Next.js.

### **Short Circuiting**

We can use `&&` and `||` operators to conditionally render components or values in React. This is known as short-circuit evaluation.

```javascript
{
  userImg && (
    <Image
      src={userImg || "/assets/images/dummy.jpg"}
      alt="User"
      width={66}
      height={66}
      className="rounded-full"
    ></Image>
  );
}
```

In the above example, if `userImg` is truthy, the `Image` component will be rendered. If `userImg` is falsy, nothing will be rendered.

Also, in the `src` attribute of the `Image` component, if `userImg` is falsy, it will default to `"/assets/images/dummy.jpg"`.

### **Using Map for Rendering Lists**

Whenever we use an `array` inside a component, it will be rendered as a list.

We can make use of the `map` function to iterate over the array and render each item as a component.

For example,

```ts
isOpen && (
          <ul className="dropdown">
            {["Most Recent", "Most Liked"].map((option) => (
              <li key={option} className="list-item">
                {option}
              </li>
            ))}
          </ul>
        )}

```

In, the above example, we are rendering a list of `options` inside a dropdown. The `map` function is used to iterate over the array of options and render each option as a list item. The `key` prop is used to uniquely identify each list item, which helps React optimize rendering.

Note,

Here, we are not use `{}` to define expression for the `callback` function of the `map` method. This is because we are returning a single expression, which is the list item therefore we are using `()`. If we were to use `{}`, we would need to explicitly return the list item using the `return` statement.

### **Creating Dummy Array**

```ts
{
  Array.from({ length: 5 }).map((_, index) => (
    <Image
      src="/assets/icons/star.svg"
      alt="star"
      width={20}
      height={20}
      key={index}
    ></Image>
  ));
}
```

In the above code, we are creating a dummy array of length 5 using `Array.from({ length: 5 })`.

This creates an array `[undefined, undefined, undefined, undefined, undefined]` which we can then map over to render our `Image` components.

The `_` is a convention used to indicate that the value is not going to be used. In this case, we are not using the first argument of the `map` function, which represents the current element of the array. Instead, we are only interested in the `index` argument, which represents the current index of the element being processed.


<hr>
<hr>
<hr>
<hr>


# **Next.js Notes**

Here are some notes related to Next.js concepts, features, and best practices for building modern web applications with Next.js.

### **Importing**

If we've to import a constant or utility from a file in the root directory, we can use the `@` alias to refer to the root directory. This allows us to avoid using relative paths like `../../constants`.

For example, if we want to import the `ICONS` object from the `constants` file, we can do it as follows:

`import { ICONS } from "@/constants";` imports the `ICONS` object from the `constants` file located in the root directory of the project. The `@` symbol is an alias for the root directory, which allows us to import files without using relative paths.

<hr>


## **Route in Next.js**

We need to define the route for the `onClick` event to change the route in Next.js. This can be done using the `useRouter` hook from the `next/navigation` package. The `useRouter` hook provides access to the router object, which allows us to navigate to different routes programmatically.

The `useRouter` hook returns an object with various methods, including `push`, `replace`, and `back`, which can be used to navigate to different routes.

**`push`** method is used to navigate to a new route, while **`replace`** method is used to replace the current route with a new one. The **`back`** method is used to go back to the previous route.

<hr>

```typescript
import { useRouter } from "next/navigation";
import React from "react";

const MyComponent = () => {
  const router = useRouter();

  const handleClick = () => {
    router.push("/new-route");
  };

  return <button onClick={handleClick}>Go to New Route</button>;
};

export default MyComponent;
```

In the above example, we import the `useRouter` hook from the `next/navigation` package and use it to navigate to the `/new-route` when the button is clicked. The `router.push()` method changes the route to the specified path.


<hr>
<hr>
<hr>
<hr>


## **Notes Related to Working of NextJs**

<hr>


Next Day understand the NextJs build process completely, we will go through the `layout.tsx` and `page.tsx` files.

**Resources**

[Link1](https://www.dhiwise.com/post/mastering-the-nextjs-build-process-for-optimal-performance)

[Link2](https://sudoshreyansh.medium.com/how-next-js-builds-your-app-6db6059ca34b)

[Link3](https://nextjs.org/docs/14/pages/building-your-application/deploying)

[Link4](https://blog.stackademic.com/understanding-the-next-js-production-build-f8d4eb23e469)

## **NextJs Build**

<hr>

When you run the `next build` command, the final build output will include the following:

**To Be Continued...**


## **Understand NextJs Layout and Page (NextJs in Deployment)**

By default, Next.js uses a file-based routing system, where each file in the `app` directory corresponds to a route in the application. The `layout.tsx` file is used to define the layout for the entire application, while the `page.tsx` file is used to define the content for the home page.

**My Question**

### 🧵 Real-World Deployment & Routing Behavior in Next.js – Am I Thinking Right?

I’ve been diving deep into how Next.js works under the hood, especially with the App Router and how layout/page rendering and navigation happen — but I just want to make sure I’ve got it right.

Let’s say we have a route like `/home`, and inside that we have nested routes like `/home/admin`, `/home/login`, etc.

From what I understand:

- The `layout.tsx` inside `app/home/` acts as the shared layout for all its child routes.
- When a user first visits `/home`, the full HTML (including the layout) is rendered on the server and sent to the browser.
- Now if they navigate to `/home/login`, since the layout hasn’t changed, Next.js won’t re-render the whole page — just the part that’s different (`page.tsx` inside `/home/login/`), right?

I’m also trying to understand what exactly happens on the network during that route change.

Is it like this?

- Instead of requesting a whole new HTML file like a traditional multi-page app, the client sends a request to the Next.js server (some API or internal endpoint?).
- That server returns some data (maybe not raw HTML, but something like a component tree or serialized JSX?).
- Then the client JS takes that, figures out what’s new, and updates the DOM inside the already-mounted layout.

Is that how it works in the real world?

Also, when we build the project (`next build`), does it generate separate HTML/CSS/JS chunks per route or component so that only what’s needed is loaded during navigation?

Lastly, I'm used to working with Express where you do something like `http.createServer(...)` — but how does that server part work in Next.js deployments? Is it managed for us (like on Vercel), or do we manually handle server setup?

<hr>

**Answer**

Excellent explanation attempt! You're very close and clearly thinking deeply about **how Next.js works under the hood**, especially with the **App Router**, **server components**, **routing**, and **deployment model**.

Let’s now walk through your scenario and provide **a full, in-depth technical breakdown** — clarifying where you're absolutely right, and where small adjustments are needed.

---

### 🧭 Scenario: You have a path `/home` and sub-paths like `/home/admin`, `/home/login`, etc.

Let’s assume you’re using the **Next.js App Router** with this folder structure:

```
app/
├── layout.tsx               ← Global root layout
├── home/
│   ├── layout.tsx           ← Layout for /home and all subroutes
│   ├── page.tsx             ← Renders /home
│   ├── login/
│   │   └── page.tsx         ← Renders /home/login
│   └── admin/
│       └── page.tsx         ← Renders /home/admin
```

---

### ✅ What Happens at Build Time (`next build`)

1. **Layouts and Pages are compiled**

- Each `layout.tsx` and `page.tsx` is compiled into a **React Server Component** (RSC).
- Next.js uses **Webpack** and **Turbopack** to bundle code per route — only the JS/CSS needed for `/home/login` is included in that route's chunk.

2. **Server code and client code are separated**

- Server components (`layout.tsx`, `page.tsx`) don’t ship to the browser.
- Only **client components** (with `'use client'`) are bundled for the browser.

3. **Font files**, images, and static files are hashed and stored under:

```
.next/static/media/
```

4. **HTML pre-rendering (if using SSG) or bundling for SSR**

- If you use `generateStaticParams` and `export const dynamic = 'force-static'`, Next.js may pre-render `.html` and `.json` files for those routes.
- Otherwise, for SSR (default), the HTML will be generated dynamically at runtime.

---

### 🛰️ Now What Happens at Runtime (after Deployment)

You’re correct that the app doesn’t work like a simple Express server doing `http.createServer()`.

Next.js applications, once deployed, behave as a **hybrid SSR framework + static server + API server**, often hosted on:

- **Node.js server (custom or Vercel’s)**
- **Edge runtimes (like Vercel Edge Functions or Cloudflare Workers)**
- **Docker / standalone deployments**

Let’s go step-by-step through a request.

---

### 📍 User visits `/home` (first time)

1. **Client makes request to `/home`**

   - The server sees this and begins resolving:

     ```
     Root layout.tsx
     └─ home/layout.tsx
        └─ home/page.tsx
     ```

2. **React Server Component Tree is rendered**

   - Server renders HTML starting from outermost layout down to the page.
   - CSS classes, `@font-face`, and metadata are injected.
   - This full HTML is streamed back to the client.

3. **Browser receives and paints HTML**

   - The HTML already contains `div#__next` with all components rendered.
   - Fonts are preloaded.
   - JS bundles are preloaded and begin hydrating.

4. **Hydration happens**

   - Next.js uses React to attach event listeners and state to already-rendered HTML.
   - This allows interactivity (buttons, forms, etc.).

---

### 📍 Then user navigates (via `<Link href="/home/login" />`)

Now comes the part you’re curious about.

### What **doesn't happen**:

> ❌ The browser doesn’t reload the page
> ❌ The browser doesn’t request a full HTML page again
> ❌ It doesn’t download a "plain text HTML file" for insertion

---

### ✅ What actually happens:

1. **The client-side router (`next/navigation`) intercepts the route change.**

   - This is like how React Router works — single-page navigation.

2. **Next.js fetches a JSON payload from the server**

   - This is **not raw HTML**, but **serialized React component output** (React Flight protocol).
   - It looks something like:

     ```json
     {
       "children": [...],
       "props": {...},
       "__type": "react.server.component.tree"
     }
     ```

3. **Client-side React receives this JSON and renders it**

   - React uses this data to rebuild the new component tree (`home/login/page.tsx`).
   - Any layouts that haven’t changed (`home/layout.tsx`, `root layout.tsx`) are **reused**.
   - Only the new part (`home/login/page.tsx`) is rendered and hydrated.

4. **The new DOM gets mounted without full page reload**

   - Browser doesn't refresh.
   - Layout doesn't re-render if it hasn't changed.
   - Only the new route content is added inside the layout.

This is why your `layout.tsx` stays persistent and only inner content changes.

---

## 🔥 Advanced Concepts Involved

| Feature            | How it Works                                                                  |
| ------------------ | ----------------------------------------------------------------------------- |
| **Streaming HTML** | Server can stream layout/page chunks gradually for better TTFB                |
| **React Flight**   | Transports server component output to client as JSON, not HTML                |
| **Server Actions** | You can run functions on the server (like mutations) directly from components |
| **Code Splitting** | Each route has its own CSS/JS chunk                                           |
| **Edge Rendering** | Optional runtime on edge functions for faster performance globally            |

---

## 🛠️ Deployment Options

You asked:

> “How is this deployed? Like `http.createServer()`?”

### ✅ No, you don’t manage `http.createServer()` manually.

Here’s how deployment usually works:

1. **Vercel (default)**

   - Automatically deploys each route as a serverless or edge function.
   - Handles routing, scaling, and caching.

2. **Custom Node Server**

   - You can use `next start` after `next build`.
   - Internally, this sets up:

     ```js
     import http from "http";
     import next from "next";
     const app = next({ dev: false });
     app.prepare().then(() => {
       http
         .createServer((req, res) => {
           app.getRequestHandler()(req, res);
         })
         .listen(3000);
     });
     ```

3. **Docker / Standalone**

   - With `output: 'standalone'` and optional custom server.
   - Bundle everything needed into `.next/standalone`.

---

### ✅ Final Verdict on Your Understanding

| Your Statement                            | Correct? | Clarification                                                                        |
| ----------------------------------------- | -------- | ------------------------------------------------------------------------------------ |
| Layout renders base HTML first?           | ✅       | Yes, it's streamed from the server                                                   |
| On navigation, we don't reload layout?    | ✅       | Correct — it’s persistent                                                            |
| Do we request plain HTML for new pages?   | ❌       | No — we request **React component tree JSON** (React Flight)                         |
| Is there one CSS/HTML file per component? | 🟡       | CSS is split per route, but HTML is rendered dynamically per layout/page             |
| Deployment like Express server?           | ❌       | You don’t write it — but under the hood, Next.js runs a smart, optimized Node server |

---

### 🧪 Want to See It in Action?

Use the browser DevTools → **Network tab** → Filter by "Doc" or "XHR":

- On initial load, you’ll see the full HTML.
- On route changes, you’ll see `.rsc` or `.json` fetches — not full page reloads.

---

<hr>

## **TL;DR**

So, when NextJs builds your app, it optimizes everything — from fonts to layouts to routes. It preloads fonts, generates CSS variables, and ensures that your app is fast and efficient. When users navigate, it uses client-side routing to update only the necessary parts of the page without reloading the entire layout, making the experience smooth and fast.

It divides the components based on routes and based on server or client components, it optimizes the loading of styles and scripts. The deployment is managed by Next.js, which handles the server setup and routing for you, allowing you to focus on building your application without worrying about the underlying infrastructure.

Also, it does not load the entire HTML file for each route change, but rather fetches the necessary data and updates the DOM accordingly, making it a highly efficient framework for building modern web applications.

It calls the API to fetch the required component as `JSON`, which is then used to update the existing layout without a full page reload.

This approach allows for a **fast, responsive user experience** while maintaining the benefits of server-side rendering and static site generation.


# **Understanding Node Runtime and Edge Runtime in Next.js**

_A Q\&A between a Senior Next.js Developer and a Curious Intern_

---

## 1. **Intern:**

So Next.js does not run on Node.js runtime? The creators changed Node.js and made their own runtime called Edge Runtime where every Next.js runs?

**Senior Dev:**
Not exactly.
By default, a Next.js application **runs on the Node.js runtime**, the same environment used by most backend JavaScript code.
The **Edge runtime** is an **additional runtime option**, not a replacement for Node.js.
It’s built on V8 isolates (like Chrome’s JavaScript engine) and works more like Cloudflare Workers or Deno Deploy, not like Node.js.

---

## 2. **Intern:**

Okay, so what’s the difference between Node runtime and Edge runtime?

**Senior Dev:**

**Node.js Runtime**

- Traditional JavaScript runtime for backend servers
- Gives full access to Node APIs like `fs`, `net`, and `crypto`
- Can connect to databases directly over TCP (e.g., using `node-postgres` or `mysql2`)
- Runs in one or a few centralized locations
- Cold starts can be slower in serverless environments

**Edge Runtime**

- Lightweight JavaScript runtime running at CDN/edge locations close to the user
- Supports only Web Standard APIs (`fetch`, `Request`, `Response`, `crypto.subtle`)
- No Node.js APIs, no TCP sockets
- Very fast startup times and globally distributed execution
- Ideal for quick tasks like authentication, caching, A/B testing, personalization

---

## 3. **Intern:**

So in Next.js, everything runs on Edge runtime by default?

**Senior Dev:**
No. In a brand-new Next.js app without any special configuration:

- **Pages, API routes, and server components** → Node.js runtime
- **Middleware** → Edge runtime (this is the only part that defaults to Edge)

Unless you explicitly tell Next.js to run certain code in Edge runtime, almost everything runs in Node.js runtime.

---

## 4. **Intern:**

So if I create a Next.js app, will it run some parts on Node runtime and some on Edge runtime automatically?

**Senior Dev:**
Not automatically. By default:

- Everything is Node.js runtime except Middleware.
- Middleware is **always** Edge runtime.
- If you want an API route or page to run on Edge, you must explicitly add:

```ts
export const runtime = "edge";
```

---

## 5. **Intern:**

Then why would a project choose Edge runtime instead of Node runtime?

**Senior Dev:**
Projects choose Edge runtime when they need:

- **Global distribution** (run code close to users around the world)
- **Low-latency responses**
- **Very fast cold starts**
- To handle lightweight logic that doesn’t need full Node APIs

The trade-off is that some libraries, like `node-postgres`, won’t work because Edge runtime doesn’t support raw TCP connections. In such cases, developers use databases that work over HTTP APIs (like Xata, Supabase REST, FaunaDB).

---

## 6. **Intern:**

So that’s why our project used Xata with Drizzle ORM instead of node-postgres?

**Senior Dev:**
Exactly. Since `node-postgres` needs TCP sockets, it won’t run in Edge runtime. Xata works over HTTP, so it’s compatible with Edge runtime.
If the team wanted to stick to Node runtime, they could have used `node-postgres` — but they probably wanted the performance benefits of Edge for some parts of the app.

---

## 7. **Intern:**

So basically, Node runtime is for full-featured backend code, and Edge runtime is for fast, distributed, lightweight code?

**Senior Dev:**
That’s the right way to think about it. They’re not replacements for each other — they’re tools for different use cases. In a single Next.js app, you can mix both depending on your needs.

---

### **Key Takeaways**

- **Node.js runtime** → Full Node API access, TCP connections, heavier server logic, runs in centralized locations.
- **Edge runtime** → Web API only, no TCP, runs globally with low latency, perfect for small tasks close to the user.
- In Next.js, **Node.js is default** except for Middleware (always Edge). You can opt into Edge manually for other routes.
- Some libraries won’t work in Edge runtime, so database choices matter.

## **Summary Table**

| Feature / Code Location | Default Runtime    | Can Switch to Edge?         |
| ----------------------- | ------------------ | --------------------------- |
| Pages (SSR)             | Node.js            | ✅ Yes (`runtime = 'edge'`) |
| API Routes              | Node.js            | ✅ Yes (`runtime = 'edge'`) |
| Middleware              | Edge               | ❌ Always Edge              |
| Static Generation (SSG) | Build-time Node.js | ❌ N/A                      |


<hr>
<hr>
<hr>
<hr>


## **`index.d.ts` File**

These types of files are used to provide type definitions for the project. It allows `TypeScript` to understand the types of various components, props, and other entities used in the project.

<hr>

The `index.d.ts` file is a TypeScript declaration file that provides type definitions for the project. It allows `TypeScript` to understand the types of various components, props, and other entities used in the project.

This file is essential for ensuring type safety and autocompletion in the development environment. It typically includes interfaces, types, and module declarations that are used throughout the project.

This file is automatically generated by `Next.js` and is used to provide type definitions for the project. It allows `TypeScript` to understand the types of various components, props, and other entities used in the project.

This file is essential for ensuring type safety and autocompletion in the development environment. It typically includes interfaces, types, and module declarations that are used throughout the project.

<hr>

### **What Happens at the Build Time?**

During the build time, Next.js compiles the TypeScript code and generates the necessary type definitions. This process ensures that the application is type-safe and that any potential type errors are caught early in the development process.

Then compile the `TypeScript` code into `JavaScript` using `Babel` or `TSC` (TypeScript Compiler). This process generates the necessary type definitions and ensures that the application is type-safe.

Also, the TypeChecking is done during the build time to ensure that the application is type-safe and that any potential type errors are caught early in the development process.

No TypeChecking is done during the RunTime, as the TypeScript code is already compiled into JavaScript. However, if you have enabled `strict` mode in your `tsconfig.json`, then the TypeScript compiler will check for type errors during the build time.


## **Routing in Next.js**

<hr>

Next.js uses a file-based routing system, where each file in the `app` directory corresponds to a route in the application. The routing is dynamic and can be nested, allowing for complex applications with multiple levels of navigation.

### **Basic Routing**

In Next.js, the routing is based on the file structure in the `app` directory. Each file represents a route, and the folder structure defines the hierarchy of routes. For example:

```
app/
├── about/
│   └── page.tsx             ← Renders /about
├── blog/
│   ├── [slug]/
│   │   └── page.tsx         ← Renders /blog/[slug]
│   └── page.tsx             ← Renders /blog
├── contact/
│   └── page.tsx             ← Renders /contact
```

### **Dynamic Routing**

Dynamic routing allows you to create routes that can change based on the data or parameters passed to them. In Next.js, you can create dynamic routes by using square brackets in the file name.

For example, if you want to create a route for blog posts that can change based on the post slug, you can create a file named `[slug].tsx` inside the `blog` directory:

```
app/
├── blog/
│   ├── [slug]/
│   │   └── page.tsx         ← Renders /blog/[slug]
│   └── page.tsx             ← Renders /blog
```

### **Nested Routing**

Nested routing allows you to create routes that are organized in a hierarchy. In Next.js, you can create nested routes by creating folders inside the `app` directory.

For example, if you want to create a route for a blog post that has comments, you can create a folder named `comments` inside the `[slug]` folder:

```app/
├── blog/
│   ├── [slug]/
│   │   ├── comments/
│   │   │   └── page.tsx         ← Renders /blog/[slug]/comments
│   │   └── page.tsx             ← Renders /blog/[slug]
│   └── page.tsx                 ← Renders /blog
```

### **Catch-All Routes**

Catch-all routes allow you to create a route that can match multiple segments of the URL. In Next.js, you can create a catch-all route by using three dots (`...`) in the file name.

For example, if you want to create a route that can match any URL under the `blog` directory, you can create a file named `[...slug].tsx` inside the `blog` directory:

```
app/
├── blog/
│   ├── [...slug]/
│   │   └── page.tsx         ← Renders /blog/[...slug]
│   └── page.tsx             ← Renders /blog
```

### **Route Groups**

Route groups allow you to organize routes into logical groups without affecting the URL structure. You can create route groups by wrapping folder names in parentheses.

For example, you can group authentication-related routes together:

```
app/
├── (auth)/
│   ├── login/
│   │   └── page.tsx         ← Renders /login
│   ├── register/
│   │   └── page.tsx         ← Renders /register
│   ├── forgot-password/
│   │   └── page.tsx         ← Renders /forgot-password
│   └── layout.tsx           ← Shared layout for auth pages
├── (dashboard)/
│   ├── analytics/
│   │   └── page.tsx         ← Renders /analytics
│   ├── settings/
│   │   └── page.tsx         ← Renders /settings
│   └── layout.tsx           ← Shared layout for dashboard pages
└── page.tsx                 ← Renders /
```

The `(auth)` and `(dashboard)` folders are route groups that:

- **Don't affect the URL structure** - `/login` not `/(auth)/login`
- **Allow shared layouts** - Each group can have its own `layout.tsx`
- **Help organize code** - Logically group related routes together
- **Enable different styling** - Apply different themes per group

Route groups are particularly useful for:

- **Authentication flows** - Group login, register, reset password pages
- **Admin sections** - Separate admin routes with different layouts
- **Multi-tenant apps** - Different layouts for different user types
- **A/B testing** - Group routes for different experimental layouts


## **Understanding Next.js Directives**

`Directives` in `Next.js` are special comments that provide hints to the Next.js `compiler` about how to handle specific files or components. They can be used to control various aspects of the build process, such as `enabling` or `disabling` certain features, specifying how components should be rendered, or providing metadata for the application.

<hr>

## **Common Next.js Directives**

### **`'use client'`**

- This directive indicates that the component should be rendered on the client side.
- It is used to enable client-side interactivity in components that are otherwise server-rendered.

```typescript
"use client";

export default function ClientComponent() {
  return <div>This component is rendered on the client side.</div>;
}
```

### **`'use server'`**

- This directive indicates that the component should be rendered on the server side.
- It is used to enable server-side rendering for components that do not require client-side interactivity.

```typescript
"use server";

export default function ServerComponent() {
  return <div>This component is rendered on the server side.</div>;
}
```

### **`'use strict'`**

- This directive enables strict mode for the component, which helps catch common errors and enforce best practices.
- It is useful for identifying potential issues in the code during development.

```typescript
"use strict";

export default function StrictComponent() {
  return <div>This component is in strict mode.</div>;
}
```

### **`'use no-cache'`**

- This directive indicates that the component should not be cached.
- It is used to ensure that the component is always rendered fresh on each request.

```typescript
"use no-cache";

export default function NoCacheComponent() {
  return <div>This component is not cached.</div>;
}
```

### **`'use dynamic'`**

- This directive indicates that the component should be rendered dynamically.
- It is used to enable dynamic rendering for components that may change frequently or require real-time updates

```typescript
"use dynamic";
export default function DynamicComponent() {
  return <div>This component is rendered dynamically.</div>;
}
```

### **`'use static'`**

- This directive indicates that the component should be rendered statically.
- It is used to enable static rendering for components that do not change frequently and can be pre
  -rendered at build time.

```typescript
"use static";
export default function StaticComponent() {
  return <div>This component is rendered statically.</div>;
}
```


<hr>
<hr>
<hr>
<hr>


# **Important Concepts to Master**


<hr>
<hr>


## **Understanding Hydration in Next.js**

Things we need to have understand before we understand hydration in Next.js:

1. **Server-Side Rendering (SSR)**: The process of rendering a web page on the server and sending the fully rendered HTML to the client.

2. **Client-Side Rendering (CSR)**: The process of rendering a web page in the browser using JavaScript after the initial HTML has been loaded.

<hr>

Now,

**What was the problem before hydration?**

Before hydration, the problem was that when a page was rendered on the server and sent to the client, it was static HTML. This meant that the page could not be interactive until the JavaScript was loaded and executed in the browser. Users would see a fully rendered page, but any dynamic behavior (like clicking buttons, submitting forms, etc.) would not work until the JavaScript was fully loaded and executed.

So, the page would appear static and unresponsive until the JavaScript was ready, leading to a poor user experience.

To solve this problem, Next.js introduced `hydration`.

### **Hydration**

We knew what was the problem before hydration, now let's understand what hydration is.

`Hydration` is the process of taking the static HTML that was rendered on the `server` and `"activating"` it in the browser by attaching `event listeners` and `state` to it. This allows the page to become `interactive` without needing to `reload` or re-render the entire page.

When a user visits a page, Next.js sends the fully rendered HTML to the `client`. Once the `JavaScript` is loaded, Next.js "hydrates" this HTML by:

1. **Attaching Event Listeners**: It adds event listeners to the elements in the HTML so that they can respond to user interactions (like clicks, form submissions, etc.).

2. **Restoring Component State**: It restores the state of the components (like form inputs, toggles, etc.) to match what was rendered on the server.

3. **Making the Page Interactive**: After hydration, the page becomes fully interactive, allowing users to interact with it as they would with a client-rendered page.

### **How Does Hydration Work in Next.js?**

When a user visits a page in a Next.js application, the following steps occur:

1. **Server Renders the Page**: The server renders the page and sends the fully rendered HTML to the client.

2. **Client Receives the HTML**: The browser receives the HTML and displays it to the user. At this point, the page is static and unresponsive.

3. **JavaScript Loads**: The JavaScript bundle for the page is loaded in the browser.

4. **Hydration Begins**: Next.js starts the hydration process by:

   - Parsing the HTML to find React components.
   - Attaching event listeners to the elements in the HTML.
   - Restoring the state of the components.

5. **Page Becomes Interactive**: Once hydration is complete, the page becomes fully interactive, and users can interact with it as they would with a client-rendered page.

### **Benefits of Hydration**

1. **Improved Performance**: By rendering the initial HTML on the server, users see a fully rendered page faster, improving the perceived performance of the application.

2. **SEO Friendly**: Since the initial HTML is fully rendered, search engines can crawl and index the content more effectively.

3. **Better User Experience**: Users can see the content immediately, and once hydration is complete, they can interact with the page without any noticeable delay.

### **Challenges with Hydration**

While hydration provides many benefits, it also comes with some challenges:

1. **Content Mismatch**: If the content rendered on the server does not match what the client expects, it can lead to warnings like "Text content did not match." This can happen if there are differences in how the server and client render components.

2. **Performance Overhead**: Hydration can add some overhead, especially for large pages with many components. The process of attaching event listeners and restoring state can take time.

3. **JavaScript Dependency**: The page relies on JavaScript to become interactive, so if the JavaScript fails to load or execute, the page will remain static.

### **Best Practices for Hydration in Next.js**

To ensure smooth hydration in Next.js applications, consider the following best practices:

1. **Consistent Rendering**: Ensure that the server and client render the same content. Avoid using browser-specific APIs or state that can lead to mismatches.

2. **Optimize Component Size**: Keep components small and focused to reduce the overhead during hydration.

3. **Minimize State**: Keep the component state to a minimum and avoid complex state management during hydration.

4. **Use `useEffect` Wisely**: Use the `useEffect` hook to handle client-specific logic that should not run during server rendering.

5. **Monitor Performance**: Use tools like Lighthouse to monitor the performance of your application and identify any hydration-related issues.

### **Conclusion**

Hydration is a crucial process in Next.js that transforms static HTML into an interactive web page. By rendering the initial content on the server and then hydrating it in the browser, Next.js provides a fast, SEO-friendly, and user-friendly experience. Understanding how hydration works and following best practices can help developers build efficient and responsive applications.
